In [47]:
from dotenv import load_dotenv
load_dotenv()


True

In [50]:
import os
PINECONE_INDEX_NAME = os.getenv('PINECONE_INDEX_NAME')
PINECONE_NAMESPACE = os.getenv('PINECONE_NAMESPACE')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

print(PINECONE_INDEX_NAME)
print(PINECONE_NAMESPACE)
print(PINECONE_API_KEY)

dur-pregnancy
dur-pregnancy-ns
pcsk_5TkRik_EzGTnXtpN1c26rjgbrhySAUqgAAHhR9QEmje4vwZkMoEKiADwUKrPjkEAQwfsTs


In [117]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def generate_prompt(query):
    prompt = ChatPromptTemplate.from_messages([
         ('system', """
        Persona: You are a knowledgeable and experienced pharmacist with a passion to help people. 
        
        Role: As a pharmacist, your role is to provide simple and clear explanations about medicines for general people.
        
        Examples:
        Your explanation should cover the following:
        
        1. A brief explanation of the medicine, its effectiveness, dosage, precautions, side effects, and storage information.
        2. If the medicine is safe for pregnant women, including any contraindications, warning levels, and what they mean.
        
        Use the context provided below to answer the user's query.
        """),
        HumanMessagePromptTemplate('''주어진 맥락에 사용하여 질문에 답변을 하되, 모르는 경우 모른다고 말하며 최대한 세문장으로 간결하게 한국말로 답변해주세요. 
        query:{query} , content:{content}
        '''),
    ])
    model = ChatOpenAI(
        model='gpt-4o',
        temperature=1
    )
    
    output_parser = StrOutputParser()
    
    pro_chain = prompt | model | output_parser
    return pro_chain

In [111]:
generate_prompt

<function __main__.generate_prompt(query)>

# 1. 임산부 금기 여부

In [120]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings


def similarity_search_pregnant(query):
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
    vector_db = PineconeVectorStore(
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME,
        namespace=PINECONE_NAMESPACE,
        pinecone_api_key=PINECONE_API_KEY
    )
    
    result = vector_db.similarity_search(
        query,
        k=3,
        namespace=PINECONE_NAMESPACE,
    )
    
    return{
        "query": query,
        "content" : '\n'.join([doc.page_content for doc in result]),
    }

In [132]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = RunnableLambda(similarity_search_pregnant)
chain.invoke('코스페낙정에 대해 말해줘')

{'query': '코스페낙정에 대해 말해줘',
 'content': '성분명: mometasone\n제품명: 나조코비나잘스프레이(모메타손푸로에이트)_(7mg/140회)\n업체명: 동광제약(주)\n금기등급: 2\n상세정보: (점비제) 임부에 대한 안전성 미확립. 임신 중 코르티코스테로이드류를 투여 받은 어머니에게서 태어난 영아는 세심하게 부신기능부전을 관찰하여야 함\n성분명: mometasone\n제품명: 코코나잘스프레이(모메타손푸로에이트)_(7mg/140회)\n업체명: 한국휴텍스제약(주)\n금기등급: 2\n상세정보: (점비제) 임부에 대한 안전성 미확립. 임신 중 코르티코스테로이드류를 투여 받은 어머니에게서 태어난 영아는 세심하게 부신기능부전을 관찰하여야 함\n성분명: mometasone\n제품명: 나조큐어나잘스프레이(모메타손푸로에이트)_(7mg/140회)\n업체명: 국제약품(주)\n금기등급: 2\n상세정보: (점비제) 임부에 대한 안전성 미확립. 임신 중 코르티코스테로이드류를 투여 받은 어머니에게서 태어난 영아는 세심하게 부신기능부전을 관찰하여야 함'}

In [134]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

chain = RunnableLambda(similarity_search_pregnant)  | RunnableLambda(generate_prompt) 

chain.invoke({'':'코르페낙정'})

TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'